## Purpose 

Predict daily smoke behavior using only contextual information.

### Inputs (X)
Day (derived from date / day)
Weather
Occasion

### Outputs (y)
Probability of smoke detection today
Likely time-of-day bucket if smoke occurs


### Constraints
- No time inputs
- Time only appears as targets
- Minimal user input (forecast-style UX)

## Imports

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, classification_report

import lightgbm as lgb
